## Baseline model

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Display the dataframe
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

### Load data

In [3]:
application_21 = pd.read_excel('data/2021.xlsx', sheet_name='Resultaten')

In [4]:
schools_21 = pd.read_excel('data/2021.xlsx', sheet_name='Klassen')

In [7]:
# print(application_21.shape)
application_21.head(2)

,Basisschool advies,Lotnummer,Geplaatst op,Positie,Voorrang/Hardheid eerste voorkeur,Voorkeur 1,Voorkeur 2,Voorkeur 3,Voorkeur 4,Voorkeur 5,Voorkeur 6,Voorkeur 7,Voorkeur 8,Voorkeur 9,Voorkeur 10,Voorkeur 11,Voorkeur 12,Voorkeur 13,Voorkeur 14,Voorkeur 15,Voorkeur 16,Voorkeur 17,Voorkeur 18,Voorkeur 19,Voorkeur 20,Voorkeur 21,Voorkeur 22,Voorkeur 23,Voorkeur 24,Voorkeur 25,Voorkeur 26,Voorkeur 27,Voorkeur 28,Voorkeur 29,Voorkeur 30,Voorkeur 31,Voorkeur 32,Voorkeur 33,Voorkeur 34
0,vmbo-t/havo,7389,Niet geplaatst,-,-,Spinoza20first - v.a. vmbo-t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,vmbo-b,7742,Niet geplaatst,-,-,Clusius College - v.a. vmbo-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Null checking

In [33]:
application_21.iloc[:,:4].isnull().sum()
# No null values in the first four columns 

Basisschool advies    0
Lotnummer             0
Geplaatst op          0
Positie               0
dtype: int64

In [34]:
# Drop columns with only nulls
# From Voorkeur 23 onwards, there is no students filling in
# df.dropna(axis=1, how='all')
application_21.dropna(axis=1, how='all', inplace=True)
application_21.columns

Index(['Basisschool advies', 'Lotnummer', 'Geplaatst op', 'Positie',
       'Voorrang/Hardheid eerste voorkeur', 'Voorkeur 1', 'Voorkeur 2',
       'Voorkeur 3', 'Voorkeur 4', 'Voorkeur 5', 'Voorkeur 6', 'Voorkeur 7',
       'Voorkeur 8', 'Voorkeur 9', 'Voorkeur 10', 'Voorkeur 11', 'Voorkeur 12',
       'Voorkeur 13', 'Voorkeur 14', 'Voorkeur 15', 'Voorkeur 16',
       'Voorkeur 17', 'Voorkeur 18', 'Voorkeur 19', 'Voorkeur 20',
       'Voorkeur 21', 'Voorkeur 22'],
      dtype='object')

In [35]:
# Fill in null values in choice columns by engineered 'None' string values, convenient to synthetic data simulation
application_21.fillna("None", inplace=True)

### School list

In [36]:
schools_21.head()

,Key,VO-school,Capaciteitsgroep,Maximale capaciteit definitieve matching
0,SvPO Amsterdam - vwo,SvPO Amsterdam,SvPO Amsterdam - vwo,16
1,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,Amsterdams Beroepscollege Noorderlicht,Amsterdams Beroepscollege Noorderlicht - v.a. vmbo-b,72
2,Amsterdams Beroepscollege Noorderlicht - vmbo-k,Amsterdams Beroepscollege Noorderlicht,Amsterdams Beroepscollege Noorderlicht - vmbo-k,110
3,Barlaeus Gymnasium - vwo,Barlaeus Gymnasium,Barlaeus Gymnasium - vwo,140
4,Berlage Lyceum - Tweetalig - v.a. vmbo-t,Berlage Lyceum,Berlage Lyceum - Tweetalig - v.a. vmbo-t,72


In [37]:
# list of schools, adding 'None' (students leave it blank)
schools = list(schools_21['Key'])
schools.append('None')
len(schools)

182

### School data per education group 

In [38]:
# Split the school data into different education groups
edu_types = application_21['Basisschool advies'].unique().tolist()
application_21['Basisschool advies'].value_counts()

vwo            2290
havo/vwo       1239
havo           1206
vmbo-t         1126
vmbo-t/havo     797
vmbo-k          618
vmbo-b          390
vmbo-b/k        312
Name: Basisschool advies, dtype: int64

### Create school choice probablity matrix (school $i$ x choice $j$)

For each education type, the sc probablity matrix is in the format of a list of (Python) lists

In [39]:
# Choice and school list
schools = list(schools_21['Key'])
schools.append('None')
edu_types = application_21['Basisschool advies'].unique().tolist()

In [41]:
# Function to generate a list of school choice probability vectors (scp matrix)
# Inputs: application_df, schools, education types
# Output: a dictionary {education type: a list of school choice probability vectors}

def generate_probability_matrix(application_df, schools: list, edu_types: str):
    p_matrix_dict = {}
    for edu_type in edu_types:

        df = application_df[application_df['Basisschool advies'] == edu_type]
        # Remove choice column that has no student fill in
        df.dropna(axis=1, how='all', inplace=True)
        choice_cols = [c for c in list(df.columns) if 'Voorkeur' in c]
        p_matrix = []

        for j in choice_cols:
            p_vector = []
            for i in schools:
                perc = len(df[df[j] == i]) / (df[j].notna().sum())
                p_vector.append(perc)
            p_matrix.append(p_vector)
        p_matrix_dict[edu_type] = p_matrix
    return p_matrix_dict

In [42]:
p_matrices = generate_probability_matrix(application_21, schools, edu_types)

In [43]:
import json
# Serialize data into file:
json.dump(p_matrices, open("school_choice_p.json", 'w' ))

## Simulating synthetic data - Baseline model

### Experiment: VWO

In [44]:
vwo_p_matrix = p_matrices.get('vwo')
len(vwo_p_matrix[0])

182

### Simulation across education types

In [45]:
# A function to update p vectors based on the previous choices:
p = [0.3, 0.0, 0.5, 0.2]
L = [(0, 'A')]

def update_p_vector(p, L):
    # input: probability vector p, list of schools previously chosen with their indexes
    # output: updated vector p
    # Algorithm: if the school is already in list L, change their corresponding probability p_ij to 0, then dividing their sum 
    # of probability equally over the rest of the vector element
    if len(L) == 0 or sum(p) == 0:
        return p
    else:
        chosen_school_index = [s[0] for s in L]
        chosen_school_p_sum = sum([p[i] for i in chosen_school_index])
        # num_zero_chance_schools = len([pi for pi in p if pi == 0.0])
        for i in range(len(p)):
            if i in chosen_school_index:
                p[i] = 0.0
            else:
                p[i] = p[i] + chosen_school_p_sum/(len(p) - len(L))
        # normalize p
        p = np.asarray(p).astype('float64')
        p = p / np.sum(p)
        return list(p)

update_p_vector(p, L)


[0.0, 0.09999999999999999, 0.6, 0.3]

In [46]:
# Generate one simulation, corresponding with a single application of a student

def baseline_single_simulation(schools: list, p_vector_list: list, edu_type: str):
    # schools: list of schools (string), e.g., ['SvPO Amsterdam - vwo', 'Amsterdams BN - v.a. vmbo-b', ...]
    # p_vector_list: a list of probability vectors, each vector corresponding to each ordered choice made by a student

    simulation_dict = {}
    simulation_dict['Basisschool advies'] = edu_type
    chosen_schools = []
    previous_choice = ''

    for i, p_vector in enumerate(p_vector_list):
        if previous_choice == 'None':
            current_choice = 'None'
        else:
            updated_p_vector = update_p_vector(p_vector, chosen_schools)
            current_choice = np.random.choice(schools, 1, p=updated_p_vector)[0]
            chosen_schools.append((schools.index(current_choice), current_choice))
        simulation_dict['Voorkeur {}'.format(i)] = current_choice
        previous_choice = current_choice
    
    return simulation_dict


In [47]:
# school_tuples = [(0, 'A'), (1, 'B'), (2, 'C'), (3, 'E')]
school_example = ['A', 'B', 'C', 'D']
p_vector_list = [[0.1, 0.2, 0.3, 0.4], [0.01, 0.02, 0.0, 0.97]]
edu_type = 'vwo'

baseline_single_simulation(school_example, p_vector_list, edu_type)

{'Basisschool advies': 'vwo', 'Voorkeur 0': 'D', 'Voorkeur 1': 'B'}

In [48]:
baseline_single_simulation(schools, vwo_p_matrix, 'vwo')

{'Basisschool advies': 'vwo',
 'Voorkeur 0': 'St. Nicolaaslyceum - vwo',
 'Voorkeur 1': 'Metis Montessori Lyceum - Coderclass of Metisprofiel vwo - vwo',
 'Voorkeur 2': 'Calandlyceum - vwo',
 'Voorkeur 3': 'St. Nicolaaslyceum - Tweetalig Onderwijs - vwo',
 'Voorkeur 4': 'Pieter Nieuwland College - vwo',
 'Voorkeur 5': 'Metis Montessori Lyceum - Technasium vwo - vwo',
 'Voorkeur 6': 'Montessori Lyceum Amsterdam - vwo',
 'Voorkeur 7': 'Vox College - v.a. havo',
 'Voorkeur 8': 'Berlage Lyceum - Tweetalig - vwo',
 'Voorkeur 9': 'Hyperion Lyceum - vwo',
 'Voorkeur 10': 'Spinoza Lyceum - Gymnasium - vwo',
 'Voorkeur 11': 'Het Amsterdams Lyceum - vwo',
 'Voorkeur 12': 'None',
 'Voorkeur 13': 'None',
 'Voorkeur 14': 'None',
 'Voorkeur 15': 'None',
 'Voorkeur 16': 'None',
 'Voorkeur 17': 'None',
 'Voorkeur 18': 'None',
 'Voorkeur 19': 'None',
 'Voorkeur 20': 'None',
 'Voorkeur 21': 'None'}

In [49]:
# A function to generate a synthetic data of the same length with the input dataset, using random choice
# Input: application data, school list
# Output: synthetic data
# Assumption: application df has the same columns with application_21, i.e., 'Basisschool advies', 'Voorkeur i'

def baseline_synthetic_school_choice(df, schools):
    edu_types = df['Basisschool advies'].unique().tolist()
    p_matrices = generate_probability_matrix(df, schools, edu_types)

    simulations = []
    for e in edu_types:
        count = len(df[df['Basisschool advies'] == e])
        p_matrix = p_matrices.get(e)
        simulations_per_edu_type = []

        for i in range(count):
            simulation = baseline_single_simulation(schools, p_matrix, e)
            simulations_per_edu_type.append(simulation)
        simulations += simulations_per_edu_type
    baseline_simulation_df = pd.DataFrame(simulations)
    return baseline_simulation_df

In [50]:
baseline_synthetic_df = baseline_synthetic_school_choice(application_21, schools)
baseline_synthetic_df.to_csv('synthetic_school_choice_2021.csv')

In [51]:
baseline_synthetic_df.sample(3)

,Basisschool advies,Voorkeur 0,Voorkeur 1,Voorkeur 2,Voorkeur 3,Voorkeur 4,Voorkeur 5,Voorkeur 6,Voorkeur 7,Voorkeur 8,Voorkeur 9,Voorkeur 10,Voorkeur 11,Voorkeur 12,Voorkeur 13,Voorkeur 14,Voorkeur 15,Voorkeur 16,Voorkeur 17,Voorkeur 18,Voorkeur 19,Voorkeur 20,Voorkeur 21
3126,havo/vwo,CSB - v.a. vmbo-t,Berlage Lyceum - Kansrijk VWO - Tweetalig - havo/vwo,Pieter Nieuwland College - plus - vwo,Calandlyceum - LOOT - v.a. vmbo-t,Bredero Mavo - v.a. vmbo-t,Over-Y - Kansrijk HAVO - vmbo-t/havo,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6964,vmbo-t,Bredero Mavo - v.a. vmbo-t,De nieuwe Havo - v.a. havo,Vossius Gymnasium - vwo,Havo de Hof - v.a. havo,Calandlyceum - vwo,Geert Groote College - v.a. havo,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4820,vwo,Metis Montessori Lyceum - Coderclass of Metisprofiel vwo - vwo,CSB - Kansrijk HAVO - vmbo-t/havo,Bredero Mavo - Tweetalig - v.a. vmbo-t,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Evaluation - Baseline model

- No duplicate choice per student
- Probability distribution close to real data
- Distribution of number of choices students fill in

In [8]:
synthetic_df = pd.read_csv('school-choice-simulation-baseline.csv')
synthetic_df.head()

,Basisschool advies,Voorkeur 1,Voorkeur 2,Voorkeur 3,Voorkeur 4,Voorkeur 5,Voorkeur 6,Voorkeur 7,Voorkeur 8,Voorkeur 9,Voorkeur 10,Voorkeur 11,Voorkeur 12,Voorkeur 13,Voorkeur 14,Voorkeur 15,Voorkeur 16,Voorkeur 17,Voorkeur 18,Voorkeur 19,Voorkeur 20,Voorkeur 21,Voorkeur 22
0,vmbo-t/havo,Berlage Lyceum - Tweetalig - v.a. vmbo-t,Hervormd Lyceum West - v.a. vmbo-t,Xplore - v.a. vmbo-t,Hervormd Lyceum West - v.a. vmbo-t,Geert Groote College - v.a. vmbo-t,CSB - v.a. vmbo-t,Calandlyceum - vmbo-t/havo,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,vmbo-t/havo,Over-Y - v.a. vmbo-t,Montessori Lyceum Amsterdam - v.a. vmbo-t,Montessori Lyceum Oostpoort - v.a. vmbo-b,Lumion - v.a. vmbo-t,Geert Groote College - v.a. vmbo-t,Montessori Lyceum Amsterdam - v.a. vmbo-t,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,vmbo-t/havo,Sweelinck College - v.a. vmbo-t,Lumion - v.a. vmbo-t,Geert Groote College - v.a. vmbo-t,DENISE - Denise TL - v.a. vmbo-t,Montessori Lyceum Amsterdam - v.a. vmbo-t,Montessori Lyceum Amsterdam - v.a. vmbo-t,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,vmbo-t/havo,Lumion - v.a. vmbo-t,IJburg College - v.a. vmbo-t,Bredero Mavo - v.a. vmbo-t,Kiem Montessori - v.a. vmbo-t,Geert Groote College - v.a. vmbo-t,Spring High - v.a. vmbo-t,Marcanti College - v.a. vmbo-t,Berlage Lyceum - Tweetalig - v.a. vmbo-t,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,vmbo-t/havo,Calandlyceum - LOOT - v.a. vmbo-t,Vinse School - v.a. vmbo-t,Montessori Lyceum Oostpoort - v.a. vmbo-b,Vinse School - v.a. vmbo-t,Spinoza Lyceum - v.a. vmbo-t,Montessori Lyceum Amsterdam - v.a. vmbo-t,Vox College - Kansrijk HAVO - vmbo-t/havo,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
